In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt

from pytorch_sklearn.neural_network.neural_network_copy import NeuralNetwork as NeuralNetworkDev
from pytorch_sklearn.neural_network.neural_network import NeuralNetwork
from pytorch_sklearn.callbacks.predefined import Verbose, History, EarlyStopping
from pytorch_sklearn.utils.progress_bar import print_progress

In [3]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 3, kernel_size=3, padding=1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        return self.conv3(x)

In [4]:
model = SimpleCNN()
optim = torch.optim.Adam(model.parameters(), lr=0.001)
crit = nn.MSELoss()

In [5]:
X = torch.randn(10, 3, 32, 32)
y = torch.randn(10, 3, 32, 32)

In [6]:
net = NeuralNetwork(model, optim, crit)

In [10]:
net.fit(
    train_X=X,
    train_y=y,
    max_epochs=50,
    callbacks=[Verbose(notebook=True)],
    metrics={'l1loss': nn.L1Loss(), 'l2loss': nn.MSELoss()},
)

Epoch 51/100
1/1 [====================] - train_loss: 0.938 - train_l1loss: 0.774 - train_l2loss: 0.938 - Time: 0.01 - ETA: 0.00
Epoch 52/100



Epoch 53/100
1/1 [====================] - train_loss: 0.933 - train_l1loss: 0.772 - train_l2loss: 0.933 - Time: 0.06 - ETA: 0.00
Epoch 54/100
1/1 [====================] - train_loss: 0.931 - train_l1loss: 0.771 - train_l2loss: 0.931 - Time: 0.00 - ETA: 0.00
Epoch 55/100
1/1 [====================] - train_loss: 0.929 - train_l1loss: 0.770 - train_l2loss: 0.929 - Time: 0.00 - ETA: 0.00
Epoch 56/100
1/1 [====================] - train_loss: 0.927 - train_l1loss: 0.769 - train_l2loss: 0.927 - Time: 0.00 - ETA: 0.00
Epoch 57/100
1/1 [====================] - train_loss: 0.924 - train_l1loss: 0.768 - train_l2loss: 0.924 - Time: 0.00 - ETA: 0.00
Epoch 58/100
1/1 [====================] - train_loss: 0.922 - train_l1loss: 0.767 - train_l2loss: 0.922 - Time: 0.00 - ETA: 0.00
Epoch 59/100
1/1 [====================] - train_loss: 0.920 - train_l1loss: 0.766 - train_l2loss: 0.920 - Time: 0.00 - ETA: 0.00
Epoch 60/100
1/1 [====================] - train_loss: 0.917 - train_l1loss: 0.765 - train_l2loss